In [1]:
from langchain_openai import OpenAIEmbeddings
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd
import os

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up connections

In [136]:
openai_api_key = getpass()

 ········


In [3]:
neo4j_password = getpass()

 ········


In [4]:
neo4j_uri = "neo4j+s://bc9f751a.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key)

# Embed topic descriptions

In [6]:
topics = gds.run_cypher("""
                        MATCH (t:Topic)
                        WHERE t.embedding IS NULL
                        RETURN t.description AS description""")

In [7]:
topics['embedding'] = embeddings.embed_documents(topics['description'].to_list())

In [8]:
topics.head()

,description,embedding
0,,"[0.015375844493788012, -0.034777941596249584, ..."
1,! — NOT expression in Cypher,"[-0.001502172470261095, 0.03528362585172272, 0..."
2,!RATED relationship,"[0.01292134665386383, 0.02400438344185642, -0...."
3,!tables command,"[0.0033105522452637954, 0.0042203221288704, 0...."
4,"""(RELATIONSHIP)"" suffix","[0.019056276849615902, 0.0011400599897678637, ..."


In [9]:
gds.run_cypher("""CREATE VECTOR INDEX topic_vectors IF NOT EXISTS 
                  FOR (t:Topic)
                  ON (t.embedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 1536,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""


In [12]:
for i in range(0, int(topics.shape[0]/1000) + 1):
    gds.run_cypher("""
        UNWIND $topicData AS d
        MATCH (t:Topic {description:trim(d['description'])})
        CALL db.create.setNodeVectorProperty(t, 'embedding', d['embedding'])""",
                   {'topicData': topics.iloc[i*1000:(i+1)*1000,:].to_dict('records')})
    if (i+1)%5 == 0:
        print(f"Finished row {(i+1)*1000}")
    

Finished row 5000
Finished row 10000
Finished row 15000
Finished row 20000
Finished row 25000
Finished row 30000
Finished row 35000
Finished row 40000
Finished row 45000
Finished row 50000
Finished row 55000
Finished row 60000
Finished row 65000
Finished row 70000


# Calculate nearest neighbors

In [149]:
g_topics, result = gds.graph.project("topics", {"Topic":{"properties": "embedding"}}, "*")

In [150]:
result

nodeProjection                        {'Topic': {'label': 'Topic', 'properties': {'embedding': {'property': 'embedding', 'defaultValue': None}}}}
relationshipProjection    {'__ALL__': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {}, 'type': '*'}}
graphName                                                                                                                                  topics
nodeCount                                                                                                                                   70954
relationshipCount                                                                                                                               0
projectMillis                                                                                                                                1745
Name: 0, dtype: object

## Check similiarty scores for some sample document pairs

In [95]:
gds.run_cypher("""
UNWIND
[
    ['Cypher MATCH clause', 'Cypher MATCH statement'],
    ['Cypher MATCH clause', 'Inline MATCH clause'],
    ['MATCH (n)-[rs*]-() syntax deprecated', 'Cypher MATCH clause'],
    ['MATCH in Cypher', 'Cypher MATCH clause'],
    ['MATCH query', 'MATCH in Cypher'],
    ['MATCH with LIMIT in Cypher', 'MATCH in Cypher']
] AS themes
MATCH (n1), (n2) WHERE n1.description = themes[0] and n2.description = themes[1] 
RETURN 
n1.description AS theme1,
n2.description AS theme2,
gds.similarity.cosine(n1.embedding, n2.embedding) AS sim""")

,theme1,theme2,sim
0,Cypher MATCH clause,Cypher MATCH statement,0.946668
1,Cypher MATCH clause,Inline MATCH clause,0.708626
2,MATCH (n)-[rs*]-() syntax deprecated,Cypher MATCH clause,0.525790
3,MATCH in Cypher,Cypher MATCH clause,0.784752
4,MATCH query,MATCH in Cypher,0.547737
5,MATCH with LIMIT in Cypher,MATCH in Cypher,0.741662


## Mutate KNN at a level below which we don't think there will be many meaningful pairs

In [151]:
knn_result = gds.knn.mutate(g_topics,
                            mutateRelationshipType = "IS_SIMILAR",
                            mutateProperty = "similarity",
                            nodeProperties = "embedding",
                            topK = 5,
                            similarityCutoff = 0.85)

In [140]:
similarity_result = gds.run_cypher("""
    CALL gds.graph.relationshipProperty.stream("topics", "similarity", ["IS_SIMILAR"])
    YIELD sourceNodeId, targetNodeId, propertyValue AS similarity
    WITH gds.util.asNode(sourceNodeId) AS source, gds.util.asNode(targetNodeId) AS target, similarity
    RETURN source.description AS source, target.description AS target, similarity
    ORDER BY similarity""")
                    

In [141]:
similarity_result[similarity_result['similarity']>0.87].head(20)

,source,target,similarity
26807,Operators in Cypher,Expressions in Cypher,0.870002
26808,Classic algorithms in Neo4j,Available procedures in Neo4j,0.870003
26809,Graph-based predictions,Predictive graph-based features,0.870004
26810,Predictive graph-based features,Graph-based predictions,0.870004
26811,localdatetime.truncate in Cypher,LOCAL DATETIME in Cypher,0.870004
26812,GDS library configuration,GDS version 2.2+,0.870004
26813,Neo4j Single Sign-On (SSO),Single Sign-On (SSO),0.870006
26814,Single Sign-On (SSO),Neo4j Single Sign-On (SSO),0.870006
26815,Analytical databases,Analytic queries,0.870006
26816,Analytic queries,Analytical databases,0.870006


# Check WCC at a reasonable similarity cutoff

In [152]:
knn_result['relationshipsWritten']

169653

In [153]:
knn_result['similarityDistribution']

{'min': 0.8499984741210938,
 'p5': 0.8546295166015625,
 'max': 0.9985771179199219,
 'p99': 0.9849662780761719,
 'p1': 0.8509101867675781,
 'p10': 0.8593254089355469,
 'p90': 0.95648193359375,
 'p50': 0.8990097045898438,
 'p25': 0.8736991882324219,
 'p75': 0.9295692443847656,
 'p95': 0.9695320129394531,
 'mean': 0.9037189434961921,
 'p100': 0.9985771179199219,
 'stdDev': 0.03569805808001403}

In [154]:
gds.wcc.write(g_topics,
              writeProperty = "wccId",
              relationshipTypes = ["IS_SIMILAR"],
              relationshipWeightProperty = "similarity",
              threshold = 0.87,
              minComponentSize=2)

writeMillis                                                                                                                                                                                                                                                                                                                                                                 1287
nodePropertiesWritten                                                                                                                                                                                                                                                                                                                                                      37971
componentCount                                                                                                                                                                                                                                                        

In [155]:
gds.run_cypher("""
MATCH (t:Topic)
WHERE t.wccId IS NOT NULL
WITH t.wccId AS id, collect(t.description) AS themes
RETURN id, size(themes) AS themeCount, themes
ORDER BY themeCount DESC
LIMIT 10""")

id  themeCount  \
0    12       27048   
1     0        6294   
2   672         213   
3   324         161   
4  4304         111   
5  6906         104   
6  3887          96   
7  1811          64   
8   535          61   
9   279          60   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# Refactor similarity relationships for use with Leiden

In [156]:
gds.graph.relationships.toUndirected(g_topics, 
                                     relationship_type = "IS_SIMILAR", 
                                     mutate_relationship_type = "UNDIRECTED_SIMILAR", 
                                     aggregation = "MAX")

inputRelationships                                                                                                                                                                                                               169653
relationshipsWritten                                                                                                                                                                                                             245224
mutateMillis                                                                                                                                                                                                                          0
postProcessingMillis                                                                                                                                                                                                                  0
preProcessingMillis                                                     

In [157]:
gds.set_database("neo4j")
relationships = gds.graph.relationshipProperties.stream(g_topics, "similarity", ["UNDIRECTED_SIMILAR"])

Undirected relationships stream once in each direction. Drop the half of relationships where targetNodeId < sourceNodeId

In [158]:
relationships = relationships[relationships['sourceNodeId']<relationships['targetNodeId']]

The relationship weights are bunched between 0.8 and 1.0. Transform them so that they range from 0.0 to 1.0.

In [159]:
gds.run_cypher("""
UNWIND $rows AS row
WITH gds.util.asNode(row['sourceNodeId']) AS source,
gds.util.asNode(row['targetNodeId']) AS target,
row['propertyValue'] AS similarity
WITH gds.graph.project("reweight_topics",
source,
target,
{relationshipType: "IS_SIMILAR",
relationshipProperties: {similarity: (similarity-0.8)/0.2}},
{undirectedRelationshipTypes: ["IS_SIMILAR"]}) AS g
RETURN g.graphName AS graphName, g.relationshipCount AS relationshipCount, g.nodeCount AS nodeCount""",
               {"rows": relationships.to_dict("records")})

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke function `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: Graph reweight_topics already exists}

In [ ]:
g_topics2 = gds.graph.get("reweight_topics")

# Test Leiden at different values of gamma

In [161]:
def test_leiden_gamma(gamma):
    property_name = f"leidenGamma{gamma}"
    try:
        gds.graph.nodeProperties.drop(g_topics2, property_name)
    except:
        pass
    gds.leiden.mutate(g_topics2,
                      mutateProperty = property_name,
                      relationshipTypes = ["IS_SIMILAR"],
                      relationshipWeightProperty = "similarity",
                      gamma=gamma)
    biggest_communities = gds.run_cypher("""
        CALL gds.graph.nodeProperty.stream("reweight_topics", $propName)
        YIELD nodeId, propertyValue
        WITH propertyValue, collect(nodeId) AS ids, count(*) as themeCount
        ORDER BY themeCount desc
        LIMIT 20
        RETURN themeCount, [id in ids | gds.util.asNode(id).description] AS themes""",
                                        {"propName": property_name})
    single_doc_communities = gds.run_cypher("""
        CALL gds.graph.nodeProperty.stream("reweight_topics", $propName)
        YIELD nodeId, propertyValue
        WITH gds.util.asNode(nodeId) as t, propertyValue AS leidenId
        MATCH (d)-[:HAS_TOPIC]->(t)
        WITH leidenId, count(distinct d) AS docCount
        WITH leidenId, CASE WHEN docCount = 1 then 1 else 0 end AS singleDocCommunity
        RETURN count(*) AS communityCount, 
        sum(singleDocCommunity) AS singleDocCommunities, 
        sum(singleDocCommunity) * 1.0 / count(*) AS singleDocCommunityPercent""",
                                           {"propName": property_name})
    return {"biggest_communities": biggest_communities, "single_doc_communities": single_doc_communities}
                                           


In [187]:
gamma_tests = {gamma: test_leiden_gamma(gamma) for gamma in [4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0, 1024.0]}

In [188]:
for gamma, result in gamma_tests.items():
    print(f"Gamma value: {gamma}")
    print(result['single_doc_communities'])

Gamma value: 4.0
   communityCount  singleDocCommunities  singleDocCommunityPercent
0            2696                    92                   0.034125
Gamma value: 8.0
   communityCount  singleDocCommunities  singleDocCommunityPercent
0            2813                    92                   0.032705
Gamma value: 16.0
   communityCount  singleDocCommunities  singleDocCommunityPercent
0            2962                    92                    0.03106
Gamma value: 32.0
   communityCount  singleDocCommunities  singleDocCommunityPercent
0            3181                    92                   0.028922
Gamma value: 64.0
   communityCount  singleDocCommunities  singleDocCommunityPercent
0            3471                    92                   0.026505
Gamma value: 128.0
   communityCount  singleDocCommunities  singleDocCommunityPercent
0            3920                    92                   0.023469
Gamma value: 256.0
   communityCount  singleDocCommunities  singleDocCommunityPercent
0  

In [210]:
for gamma, result in gamma_tests.items():
    print(f"Gamma value: {gamma}")
    print(result['biggest_communities']['themeCount'].head())

Gamma value: 4.0
0    950
1    752
2    624
3    360
4    360
Name: themeCount, dtype: int64
Gamma value: 8.0
0    648
1    571
2    548
3    350
4    332
Name: themeCount, dtype: int64
Gamma value: 16.0
0    518
1    349
2    275
3    268
4    232
Name: themeCount, dtype: int64
Gamma value: 32.0
0    252
1    181
2    179
3    170
4    163
Name: themeCount, dtype: int64
Gamma value: 64.0
0    122
1    119
2    119
3    118
4    116
Name: themeCount, dtype: int64
Gamma value: 128.0
0    86
1    81
2    68
3    68
4    66
Name: themeCount, dtype: int64
Gamma value: 256.0
0    49
1    49
2    47
3    46
4    45
Name: themeCount, dtype: int64
Gamma value: 512.0
0    36
1    33
2    32
3    31
4    30
Name: themeCount, dtype: int64
Gamma value: 1024.0
0    21
1    20
2    20
3    19
4    19
Name: themeCount, dtype: int64


In [211]:
pd.set_option('display.max_colwidth', None)

## Examine the largest clusters at various gamma values

In [212]:
gamma_tests[32.0]['biggest_communities']

,themeCount,themes
0,252,"[Role in Cypher, skip in Cypher, sub in Cypher, write in Cypher, in Cypher, jobId in Cypher, job type in Cypher, post-process in Cypher, AND in Cypher, DOWN in Cypher, second in Cypher, WHEN in Cypher, THEN in Cypher, END in Cypher, ""abs in Cypher"", OUT in Cypher, MENTORS in Cypher, Cypher in Cypher, Person in Cypher, Dropped with Cypher, age in Cypher, expression in Cypher, ONLINE in Cypher, Label: Person in Cypher, released in Cypher, direction in Cypher, out-degree in Cypher, Location in Cypher, p90 in Cypher, Replaced in Cypher, CEILING in Cypher, GET in Cypher, division in Cypher, Division in Cypher, collect in Cypher, http in Cypher, Rels in Cypher, Inlined in Cypher statements, Collect in Cypher, asin in Cypher, IP address in Cypher, Prefix in Cypher, Existence in Cypher, SHOW SETTING in Cypher, 0-9 in Cypher, penalty in Cypher, Apply in Cypher, Repeat in Cypher, Side-effects in Cypher, ""Charlie"" in Cypher, writing Cypher, TEXT in Cypher, traverse in Cypher, Log in Cypher, BRIEF in Cypher, road in Cypher, Dashes in Cypher, Cypher result encapsulation, source in Cypher, done in Cypher, Cypher, relationshipsWritten in Cypher, DEP in Cypher, stdev in Cypher, stDevP function in Cypher, Sleep in Cypher, rel in Cypher, title in Cypher, description in Cypher, SET in Cypher, short description in Cypher, year of birth in Cypher, acting roles in Cypher, WITH HEADERS in Cypher, FAMILY in Cypher, best split in Cypher, route in Cypher, CREATED in Cypher, Connect in Cypher, IN TRANSACTIONS in Cypher, Reading patterns in Cypher, AS COPY OF in Cypher, READ in Cypher, INCOMING in Cypher, SemiApply in Cypher, internal activity in Cypher, FLOAT in Cypher, top five ordering in Cypher, FOLLOWS in Cypher, list deconstruction in Cypher, Writing in Cypher, grants in Cypher, WRITE in Cypher, Equality in Cypher, DIRECTED in Cypher, > in Cypher, latitude in Cypher, rightmost characters in Cypher, Called in Cypher, Distance in Cypher, ...]"
1,181,"[properties on nodes, sets node properties, String type node properties, Returning node properties, Calculate node properties, Based on node properties, comparing node properties, algorithms for node properties, Based on other node properties, numeric node properties, Scale node properties, Defining node properties, Unique properties on node, Comparing nodes, Configured node properties, Specifying node properties, compute node properties, Graph node properties, Renaming node properties, Add node properties, Write node properties, Streams node properties, write node properties, written node properties, streaming node properties, Writes node properties, writeNodeProperties config, Normalizing node properties, ignore node properties, Select node properties, Predict node property values, Generate node properties, stream node properties procedure, compress node properties, stream node properties, writeNodeProperties function, writing node properties, Deriving node properties, Index on node property, sets node property, Unique node property, adding node properties, Additional node properties, configurations for node properties, Query node properties, Setting a property on a node, Accessing node property, Added property for nodes, Supported node property types, Optional node properties, Transforming node properties, Adding node properties, Removing node properties, Encoding node properties, binary node properties, Compute node properties, Loading node properties, Set node properties, Patient node properties, Target node properties, Uniform node properties, Drop node properties, Incorporate node properties, project node properties, Incrementing node properties, Scaled node properties, Unspecified node properties, hide properties or nodes, Training without node properties, Set property, virtual node properties, New node properties, real node properties, distinct node property values, Numeric node properties, Set JSON property on node, Remove node properties, Access to node properties

In [213]:
gamma_tests[64.0]['biggest_communities']

,themeCount,themes
0,122,"[Node specification in Cypher, endNode in Cypher, Virtual Nodes in Cypher, Add nodes in Cypher, --nodes in Cypher, Delete nodes in Cypher, node in Cypher, Field nodes in Cypher, Connecting nodes in Cypher, Identify specific unconnected nodes in Cypher, Updating nodes in Cypher, Nodes created in Cypher, Updating nodes in Neo4j, Lists of nodes in Cypher, Merging nodes in Cypher, Extract node in Cypher, Excluding nodes in Cypher, Product nodes in Cypher, target node in Cypher, Node connections in Cypher, Categorize in Cypher, Node classification in Cypher, AllNodesScan Query, AllNodeScan in Cypher, AllNodesScan, Finding nodes in Cypher, Match any node in Cypher, nodeStatements in Cypher, connects nodes in Cypher, node prediction in Cypher, node lookup in Cypher, Nodes in Cypher, nodes in Cypher, NODES in Cypher, sourceNodeId in Cypher, Read nodes in Cypher, Node in Cypher, sourceNodes in Cypher, -nodes parameter in Cypher, Node classification Cypher documentation, Starting node in Cypher, Nodes function in Cypher, targetNode in Cypher, node_property in Cypher, All Nodes Scan in Cypher, Node definition in Cypher, nodes(path) in Cypher, nodes() in Cypher, NODE() function in Cypher, Collapse Nodes in Cypher, Source node in Cypher, nodes(3) in Cypher, Nodes and labels in Cypher, Movie nodes in Cypher, sourceNode in Cypher, FOR nodes in Cypher, nodes() function in Cypher, Person nodes in Cypher, Person node in Cypher, nodes function in Cypher, Leftmost node in Cypher, node weights in Cypher, NODE in Cypher, Related nodes in Cypher, ""node"" in Cypher, virtual node in Cypher, sequence in Cypher, Improved Cypher chain, Selecting Order nodes in Cypher, ANY NODE in Cypher, Sales nodes in Cypher, Engineering nodes in Cypher, missing nodes in Cypher, `nodes()` function in Cypher, Class weights in Cypher, CONNECTED_TO in Cypher, startNode() function in Cypher, NODE UNIQUE in Cypher, Adjacent nodes in Cypher, Adding Class nodes in Cypher, $deletedNodes in Cypher, end node in Cypher, Source node name in Cypher, path of nodes in Cypher, Get all nodes in Cypher, Returning nodes in Cypher, actedIn.nodes.cypher, End nodes in Cypher, NODE? in Cypher, topN in Cypher, source node in Cypher, List Concatenation in Cypher, node creation in Cypher, Reaction nodes in Cypher, Connected nodes in Cypher, endNode() in Cypher, Disconnected nodes in Cypher, AllNodesScan in Cypher, Start and end nodes in Cypher, Linking entities in Cypher, ...]"
1,119,"[Customer nodes, Candidate nodes, Original nodes, k unique nodes, Uniquely identifiable nodes, Super nodes, Super Nodes, red nodes, green nodes, green nodes validated, Unique nodes, Filter nodes, relevant nodes, Indexed nodes, real nodes, Unique lists of nodes, Ordered list of nodes, Parent nodes, Root node, list of nodes, path of nodes, Specific nodes, Collection of nodes, List of documents to nodes, See all nodes, list of target nodes, Sets of nodes, Node feature information, node features, Distinct nodes, information about nodes, distinct nodes, Node nodes, Limited information on nodes, reachable nodes, super nodes, missing nodes, Returns nodes individually, returns nodes, aggregation of nodes, Person nodes, Subset of nodes, Intermediate nodes, processed nodes, Tag nodes, classified nodes, DISTINCT Nodes, Green nodes, Stage nodes, 'k' nodes, Standin nodes, non-existent nodes, target nodes, Important nodes, composite nodes, subcategory nodes, Components of nodes, relevant nodes identification, Identifier nodes, Primary nodes, Stop nodes, node elements, anchoring nodes, neighbor nodes, Adjacent nodes set, targetNodes parameter, Target nodes input, Database nodes, Checkpoint nodes, Anchor nodes, Adjacent nodes, collapse intermediate nodes, Section nodes, Subject nodes, Entity nodes, Entities as nodes, Entity node type, Page nodes, iterate through nodes, visited nodes, restrict nodes returned, Restricting nodes returned, Visited nodes count, Visited nodes order, Leaf nodes, List of target 

In [214]:
gamma_tests[128.0]['biggest_communities']

,themeCount,themes
0,86,"[GDS admins, GDS enabled feature, GDS community edition limitations, GDS library limitations, GDS integration, GDS client installation, GDS compatibility, GDS schema, Relationship properties in GDS, gds.graph.exists, GDS Graph Exists capabilities, GDS logging capabilities, GDS edition information, GDS new approach, GDS workloads, server-side GDS, Using GDS, Benefits of GDS, GDS users, GDS software, GDS administrator, GDS supports multiple users, Users familiar with GDS, Access GDS resources, GDS CE usage, GDS technique, GDS resources, GDS graph algorithms, GDS model discovery, Regression in GDS, Scaling with GDS, using GDS, GDS algorithms completion, Overview of GDS, GDS Fundamentals, GDS representation, Projecting into GDS, Common usage patterns in GDS, GDS library version 2.0, GDS preserves parallel relationships, GDS Algorithm Performance, GDS client, GDS license key, GDS support, GDS usage, standalone GDS database, GDS application, GDS export, Data into GDS server, Enable GDS, GDS, Recent version of GDS, GDS library write functionality, Training methods in GDS, gds-guide, Machine learning in GDS, gds-resources file, gds object, GDS Community Edition, GDS requirements, GDS EE license, gds.version procedure, GDS procedure, mapping GDS syntax, Removed in GDS v2, GDS capabilities, GDS availability, GDS support for 4.x and 5.x, Sending messages in GDS, leveraging GDS, GDS numeric properties support, GDS write back, Core operations in GDS, GDS EE features, GDS Community Edition installation, No GDS algorithms, GDS Model Catalog operations, GDS on Composite database, GDS manual reference, gds prefix, Load data into GDS, Running GDS, GDS Flight Server, GDS aggregate parallel relationships, gds.version() method, Similarity in GDS]"
1,81,"[Customer nodes, Candidate nodes, initial set of nodes, Original nodes, k unique nodes, Uniquely identifiable nodes, Super nodes, Super Nodes, red nodes, green nodes, green nodes validated, Unique nodes, Filter nodes, relevant nodes, Indexed nodes, real nodes, Parent nodes, Root node, Specific nodes, Node feature information, node features, Distinct nodes, information about nodes, distinct nodes, Node nodes, Limited information on nodes, reachable nodes, super nodes, missing nodes, Person nodes, Subset of nodes, intermediate nodes, Intermediate nodes, Intermediary node, processed nodes, Tag nodes, classified nodes, DISTINCT Nodes, Green nodes, 'k' nodes, non-existent nodes, Important nodes, composite nodes, subcategory nodes, Components of nodes, relevant nodes identification, Identifier nodes, Primary nodes, node elements, anchoring nodes, neighbor nodes, Adjacent nodes set, Database nodes, Anchor nodes, Adjacent nodes, collapse intermediate nodes, Section nodes, Subject nodes, Entity nodes, Entities as nodes, Entity node type, Page nodes, List of target nodes, Groups of nodes, Nodes created, Visited nodes, Device nodes, Patient nodes, Objects as nodes, Product nodes, Document nodes, database node, Student nodes, Location nodes, End nodes reachable, Product node, Super node concept, starting node concept, Super node issues, Explicitly created nodes, Airport location nodes]"
2,68,"[labels in list, labelField String type, labelField, Rename labels, types, properties, labels and types, Applicable to multiple labels, Multiple labels, Multiple labels handling, Label assignment, Label names, Using labels, Using labels as indexes, Using labels in models, Apply labels later, Overloading labels, Properties based on labels, Product labels, Specific label usage, Labels usage, preferred label usage, Specified label, Custom labels in projection, Custom labels, Set Labels, Remove Labels, Labels limit, specified label and properties, missing label-field, Multi-label mode, Multi-label classification model, multi-label mode, Different labels, determining labels, Selection of labels, Specifying labels, set of labels, Explicit types and labels, Specify field for labels, Field label, Species 

In [215]:
gamma_tests[256.0]['biggest_communities']

,themeCount,themes
0,49,"[distinct nodes and relationships, Query Nodes with Most Relationships, Nodes without relationships, grouping nodes and relationships, Handling nodes and relationships, Limited nodes and relationships, returns all distinct nodes and relationships, Projecting nodes and relationships, Multiple relationships between two nodes, Distinct nodes and relationships, Extracting nodes and relationships, Nodes and relationships export, export nodes and relationships, Nodes with more than 2 relationships, Checking nodes and relationships, Collapse nodes to relationships, Indexed nodes and relationships, Nodes with no incoming relationships, database nodes and relationships, Anonymous nodes and relationships, Alternating nodes and relationships, Separate nodes for relationships, Index nodes and relationships, Indexing nodes and relationships, index nodes and relationships, Virtual nodes and relationships, Query nodes and relationships, exports given nodes and relationships, Extracting data from nodes, Utilizing node and relationship types, Virtual Nodes, Delete nodes and relationships, relationships to missing nodes, importing nodes, Importing nodes with labels, Attributes in relationships, Attributes of nodes and relationships, Utilize node and relationship types, Virtual Nodes & Relationships, Load nodes for relationship, No nodes or relationships reference, collapse nodes into relationships, Limits on node and relationship counts, Split between nodes and relationships, returns distinct nodes and relationships, Import nodes and relationships, 34 Billion Nodes & Relationships, Grouping nodes by attributes, nodes, relationships, and maps]"
1,49,"[apoc.coll.reverse function, apoc.coll methods, apoc.coll, apoc.coll.set, apoc.coll.elements, apoc.coll.disjunction, apoc.coll.different, apoc.coll.set procedure, apoc.coll.avg function, apoc.coll.stdev function, apoc.coll.combinations function, apoc.coll.occurrences function, apoc.coll function, apoc.coll.pairs, apoc.coll.contains function, apoc.coll namespace, apoc.coll.indexOf, apoc.coll.different function, apoc.coll.subtract, apoc.coll.pairs function, apoc.coll.elements function, apoc.path.elements, apoc.coll.elements query, apoc.coll.fill function, apoc.coll.set function, apoc.coll.flatten function, apoc.coll.intersection function, apoc.coll.frequencies function, apoc.coll.frequenciesAsMap, apoc.coll.insert function, apoc.coll.min function, apoc.coll.partition function, apoc.coll.min, apoc.coll.indexOf procedure, apoc.coll.subtract procedure, apoc.coll.toSet function, apoc.coll.zip, apoc.coll.split, apoc.text.indexOf function, apoc.coll.stdev, apoc.coll.fill, apoc.coll.frequencies, apoc.coll.flatten, apoc.coll.sortMulti, apoc.coll.sum, apoc.coll.duplicates, apoc.coll.max, apoc.label.exists procedure, apoc.coll.elements Procedure]"
2,47,"[index updates, cached updates in pagecache, update statement, parallel update statements, Node updates, sequential updates, trigger update, Database updates, database updates, Automatic index updates, Index updates triggering, Update triggers, index updating in background, Syntax update, Index updates percentage for sampling trigger, Index sampling update percentage, Block dynamic updates, Batch updates, Update actions, Large updates handling, Continuous update, Time-based updates, Database updates prevention, Database update validation, Batch large updates, large updates, update nodes, Parallel updates, Single thread updates, Frequency of updates, Concurrent updates, Trigger update, Syntax updates, Prevent concurrent modifications, Concurrent modification issues, Concurrent updates prevention, Writing Update data, Database releases, Executing update commands, Transactional updates, Database modification, Database upgrade, database runtime updates, database changes impact, Updating queries, Transaction updates, Database updates communication]"
3,46,"[Returns Graph object, Retrieve graph information, Restore graph, returned graph, return

In [216]:
gamma_tests[512.0]['biggest_communities']

,themeCount,themes
0,36,"[Domain-specific ontologies, Matching ontologies, Using ontologies, Department level ontologies, Enterprise level ontologies, Schema and ontologies, URL for Ontology, Managing ontologies, Combining Ontologies, Extending ontologies, Enriching with ontologies, External ontologies integration, Handling large Ontologies, managing multiple ontologies, Schema and ontology, Using ontologies and taxonomies, Value of ontologies and taxonomies, Combining ontologies, Semantics and ontologies, Defining ontologies, Web Protege for ontologies, Overlaying ontologies, Graph ontology management, Learning ontology from data, Making data smarter with ontology, importing RDF ontologies, Importing RDF ontologies, Extending ontology, Loading public ontologies, Matching entities to ontology, overlaying multiple ontologies, Semantic ontologies, Automating tasks with ontologies, Extracting ontology, Semantic definitions in ontology, Formalizing knowledge with ontologies]"
1,33,"[deprecated procedures, Deprecated procedures, Updated procedures and functions, expand procedures, restore procedures, Predict procedures, import procedure, procedures and functions, Blocking procedures, public procedures, Deprecated procedure, deprecated procedure, Enable procedures, works as procedure, import procedures, Removed procedures, configuration procedures, export procedures, Generate procedures, .write procedures, Generating procedures, built-in procedures, Generated procedure name, Enable necessary procedures, custom procedures, Best practice for procedures, Built-in procedure, Export procedures, List available procedures, Import procedure, SHOW PROCEDURES for available procedures, Officially supported procedures, Restore operations]"
2,32,"[Additional settings, Optional settings, Config settings, Configurable settings, connection settings, configuration settings overview, Configuration settings variation, Configuration settings removed, Configuration settings at runtime, Configuration settings reference, Configure default settings, Configuration settings moved, Upgrade Settings, configuration settings, Configuration settings, parameter configurations, Configuration Settings, effective settings, required settings, directory settings, import configuration settings, Properties setting, other settings, query configuration settings, Configuration settings overview, Configuration, Override settings, Description of settings, Import settings, Configuration settings unordered, configuration settings scopes, Runtime configuration changes]"
3,31,"[remaining relationships, Handling of relationships, Relationships on relationships, Relationships collection, Storing relationships, No matching relationships handling, Adding new relationships, PUT method for relationships, One relationship, maintaining relationships, required for relationships, --relationships option, Relationships file, People relationships, NEXT relationships, Relationships order, missing relationships, relationships written, ""relationships"", Relationships management, iterating over relationships, Define relationships, Relationships results, information about relationships, Relationships between people, positive relationships, Character relationships, context of relationships, relationships, Relationships, Self-relationships]"
4,30,"[Enable metrics, heap metrics, Collecting metrics, Compute metrics, Device metrics collection, Inserting metrics, Continuous metrics collection, Retrieving metrics, Monitoring graph metrics, Define metrics, Extracting metrics, extract metrics, collected metrics, Custom metrics in graphs, Server metrics configuration, Enable metrics logging, Server metrics prefix, Transaction metrics, Checkpoint metrics, Instance metrics, Device metrics, Expose metrics, expose metrics, Calculates specified metrics, Persist logs and metrics, Expose metrics with NOM, Endpoints and metrics, network metrics, Viewing metrics, Output of metrics]"
5,29,"[Backup support, Backup pa

In [217]:
gamma_tests[1024.0]['biggest_communities']

,themeCount,themes
0,21,"[Hierarchical clustering, Advanced clustering, Clustering, Advanced Clustering, advanced clustering, Clustering documentation, clusterings, clustering algorithm, Clustering environments, Clustering in Enterprise, Clustering architecture, database clustering, Autonomous clustering, clustering documentation, Clustering problems, final clustering, Clustering determination, Flow clustering, node clustering, Clustering stability, DBMS clustering]"
1,20,"[scalability, Vertical scaling, Graph scalability, Scalability improvements, Efficiency and Scalability, Scalability issues, Production scaling challenges, Database scalability issues, Scalability, Scalable architectures, Scalability considerations, Scalability in graphs, Scalable manner, Highly scalable, algorithm scalability, Scalability and efficiency, Scalability solutions, Scalability in data science, scalability in databases, Read scalability]"
2,20,"[Large networks, Networks, network addresses, Network relationships, Network features, network features, Network connections, network connections, Network connectivity, Network connectivity evaluation, network, List all network connections, accepted network connections, Users' network connections, Open network connections, Router connections, Block network connections, Network conditions, establish connections, Station connectivity]"
3,19,"[JSON format data, JSON format for relationships, JSON format in databases, JSON format output, JSON-formatted list, JSON output interpretation, JSON format for data, Json format conversion, JSON formatting, JSON format, JSON representation, JSON structure, JSON document format, JSON result format, JSON format data integration, JSON format for query log, JSONL format, JSON format for nodes, JSON-like dump]"
4,19,"[gds.fastRP.stats.estimate, gds.wcc.stats.estimate, gds.alpha.sllpa.stats.estimate, gds.bellmanFord.stats.estimate, gds.betweenness.stats.estimate, gds.bfs.stats.estimate, gds.knn.stats.estimate, gds.triangleCount.stats.estimate, gds.beta.k1coloring.stats.estimate, gds.kcore.stats, gds.wcc.stats, gds.alpha.sllpa.stats, gds.beta.kmeans.stats, gds.labelPropagation.stats.estimate, gds.randomWalk.stats.estimate, gds.beta.k1coloring.stats, gds.alpha.nodeSimilarity.filtered.stats, gds.alpha.hits.stats, gds.alpha.modularity.stats]"
5,19,"[sum operator, avg operator, boolean operators, DropConstraint operator, Referencing operators, grouping operators, Argument operator, Right-hand side, Left-hand side operator, Left operator, Limit operator, Skip operator, Optional operator, Distinct operator, Distinct operator optimization, Merge operator, Delete operator, Extended set of logical operators, null logical operators]"
6,18,"[Encrypted connections, Connection encryption, Encrypted network communication, TLS encrypted, HTTPS encryption, Network communications encryption, Encryption via StartTLS, Enable encryption for https, SSL enforced, StartTLS encryption, Connection encryption options, Encryption (at rest, in transit), Data encryption, Encryption in security, Setting up encryption, Encrypted at rest, SSL encryption, Server-side encryption]"
7,18,"[Proxy routing, request routing, Configure routing rules, Adding routing rules, Request routing, Configuration for routing, Query routing, Query routing decisions, Querying routes, Dynamic routing strategies, Automatic routing, Traffic routing, Proxy setup, Route read queries, Add routing rule, Server-policy routing, Routing purposes, Routing strategies]"
8,18,"[Persistent Volume Claims (PVC), Persistent volume, Persisting data using Volumes, Persistent Volume, Persistent volume for plugins, persistent volume, persistent volume for backups, Persistent volumes, Persistent Volume Selector, Persistent volume creation, Persistent volume disks, Persistent Volume Claim (PVC), PersistentVolumeClaim object, Persistent Volume Claims in Cypher, Persistent Volume (PV) in Neo4j, configure persistent volume, Persistent volume selector confi

# See what topics drop out from a cluster at various gamma values

In [218]:
def get_cluster_for_theme(property_name, topic):
    topic_df = gds.graph.nodeProperties.stream(g_topics2,
                                               [property_name],
                                               db_node_properties = ["description"],
                                               separate_property_columns=True
                                              )
    topic_cluster_id = topic_df.loc[topic_df['description']==topic, property_name].iloc[0]
    return topic_df.loc[topic_df[property_name]==topic_cluster_id, "description"].to_list()

In [219]:
virtual_relationships_64 = get_cluster_for_theme("leidenGamma64.0", "Virtual relationships")
virtual_relationships_64

['Relationships in Neo4j',
 'Relationship handling in Neo4j',
 'relationship creation in Neo4j',
 'Inverted relationships in Neo4j',
 'Relationship creation in Neo4j',
 'Relationships limitations in Neo4j',
 'existing relationships in Neo4j',
 'nodes and relationships in Neo4j',
 'Node relationships in Neo4j',
 'New relationships in Neo4j',
 'relationship management in Neo4j',
 'Nodes and relationships in Neo4j',
 'Incoming relationships in Neo4j',
 'Outgoing relationships in Neo4j',
 'Connecting relationships in Neo4j',
 'Type relationship in Neo4j',
 'relationship patterns in Neo4j',
 'Friend relationships in Neo4j',
 'Concepts and relationships in Neo4j',
 'Establishing relationships in Neo4j',
 'Contains relationship in Neo4j',
 'Naming relationships in Neo4j',
 'Creator relationship in Neo4j',
 'Authoring relationship in Neo4j',
 'Object-entity relationship in Neo4j',
 'Subcategory relationships in Neo4j',
 'Recreating relationships in Neo4j',
 'Directed relationships in Neo4j',
 

In [220]:
virtual_relationships_128 = get_cluster_for_theme("leidenGamma128.0", "Virtual relationships")
[topic for topic in optional_match_64 if topic not in optional_match_128] 

['subquery execution',
 'Subquery expressions',
 'Top-level subquery',
 'Subquery execution',
 'Subquery evaluation',
 'Subqueries execution order',
 'Subquery executions',
 'Subquery execution failure',
 'Subqueries execution',
 'Execute subqueries',
 'Returning subqueries',
 'unit subqueries',
 'Subquery modification',
 'Subquery variables',
 'Subquery RETURN clauses',
 'CALL subquery differences',
 'COLLECT subqueries']

In [221]:
virtual_relationships_256 = get_cluster_for_theme("leidenGamma256.0", "Virtual relationships")
[topic for topic in virtual_relationships_128 if topic not in virtual_relationships_256] 

['Writing relationships to Neo4j',
 'Neo4j relationship types',
 'Nodes and relationships extraction',
 'Counting relationships in Neo4j',
 'Relationships persisted natively in Neo4j',
 'Relationships in Neo4j',
 'Relationship handling in Neo4j',
 'relationship creation in Neo4j',
 'Inverted relationships in Neo4j',
 'Relationship creation in Neo4j',
 'Relationships limitations in Neo4j',
 'Node relationships in Neo4j',
 'New relationships in Neo4j',
 'Nodes and relationships in Neo4j',
 'Outgoing relationships in Neo4j',
 'Type relationship in Neo4j',
 'relationship patterns in Neo4j',
 'Friend relationships in Neo4j',
 'Concepts and relationships in Neo4j',
 'Contains relationship in Neo4j',
 'Creator relationship in Neo4j',
 'Directed relationships in Neo4j',
 'Describing relationships in Neo4j',
 'Describing relations in Neo4j',
 'Direct relationships in Neo4j',
 '"Broader than" relationship in Neo4j',
 'Interlocking relationships in Neo4j',
 'Association in Neo4j',
 'child relatio

In [222]:
virtual_relationships_512 = get_cluster_for_theme("leidenGamma512.0", "Virtual relationships")
[topic for topic in virtual_relationships_256 if topic not in virtual_relationships_512] 

['relationship management in Neo4j',
 'Directs relationship in Neo4j',
 'relationship file in Neo4j',
 'Actor and movie relationships in Neo4j',
 'Acted in relationship in Neo4j',
 'virtual entities in Neo4j',
 'relationships in Neo4j']

In [223]:
virtual_relationships_1024 = get_cluster_for_theme("leidenGamma1024.0", "Virtual relationships")
[topic for topic in virtual_relationships_512 if topic not in virtual_relationships_1024] 

['Friend relationships in Neo4j', 'Direct relationships in Neo4j']

In [224]:
virtual_relationships_1024

['Acts-In relationship in Neo4j',
 'Relation expression in Neo4j',
 'Virtual relationships',
 'Virtual Relationships in Neo4j',
 'Directed relationships in Neo4j',
 'Counting relationships in Neo4j']

In [225]:
backup_128 = get_cluster_for_theme("leidenGamma128.0", "Backup management")
backup_128

['Backup support',
 'Backup parameters configuration',
 'backup configuration',
 'Backup parameters',
 'backup status',
 'backup data',
 'Backup success',
 'Backup management',
 'Back-up procedure',
 'Backup run',
 'Backup strategies',
 'Backup procedures',
 'Backup planning',
 'Backup not supported',
 'Backup plans for coding',
 'Backup folder management',
 'Backup configuration',
 'Backup process settings',
 'Backup modes',
 'Backup options',
 'Backup server',
 'Backup client',
 'Backup volume',
 'Backup failure',
 'Broken backup risk',
 'Backup command behavior',
 'Backup problems',
 'Backup clients',
 'Backup storage solutions',
 'Backup',
 'Backup failure handling']

In [226]:
backup_256 = get_cluster_for_theme("leidenGamma256.0", "Backup management")
[topic for topic in backup_128 if topic not in backup_256] 

['Broken backup risk']

In [227]:
backup_512 = get_cluster_for_theme("leidenGamma512.0", "Backup management")
[topic for topic in backup_256 if topic not in backup_512] 

['Backup']

In [228]:
backup_1024 = get_cluster_for_theme("leidenGamma1024.0", "Backup management")
[topic for topic in backup_512 if topic not in backup_1024]

['Backup strategies',
 'Backup procedures',
 'Backup planning',
 'Backup not supported',
 'Backup plans for coding',
 'Backup modes',
 'Backup options',
 'Backup support',
 'Backup success',
 'Back-up procedure',
 'Backup run',
 'Backup failure',
 'Backup command behavior',
 'Backup problems']

In [230]:
backup_1024

['Backup folder management',
 'Backup configuration',
 'Backup process settings',
 'Backup server',
 'Backup client',
 'Backup volume',
 'Backup parameters configuration',
 'backup configuration',
 'Backup parameters',
 'backup status',
 'backup data',
 'Backup management',
 'Backup clients',
 'Backup storage solutions',
 'Backup failure handling']

# Write Leiden results to Neo4j

In [231]:
gds.set_database("neo4j")

In [232]:
gds.graph.nodeProperties.write(g_topics2, {"leidenGamma512.0": "leidenId"})

writeMillis                      459
graphName            reweight_topics
nodeProperties            [leidenId]
propertiesWritten              38152
Name: 0, dtype: object

In [233]:
gds.run_cypher("""CREATE CONSTRAINT topic_group_node_key IF NOT EXISTS FOR (n:TopicGroup) REQUIRE n.id IS NODE KEY""")

""


In [234]:
gds.run_cypher("""
MATCH (t:Topic)
WHERE t.leidenId is not null
WITH t ORDER BY t.description
WITH t.leidenId AS id, collect(t) AS topicNodes, collect(t.description) AS descriptions
MERGE (g:TopicGroup {id:id}) 
SET g.descriptions = descriptions
WITH g, topicNodes
FOREACH (t in topicNodes | MERGE (t)-[:IN_GROUP]->(g))
""")

""


In [235]:
gds.run_cypher("""
MATCH (g:TopicGroup)
WITH max(g.id) AS maxId
MATCH (t:Topic)
WHERE NOT EXISTS {(t)-[:IN_GROUP]->()}
AND COUNT {(t)<-[:HAS_TOPIC]-()} > 1
WITH maxId, COLLECT(t) AS needsGroup
UNWIND range(0, size(needsGroup)-1) AS idx
WITH maxId, idx, needsGroup[idx] AS t
MERGE (g:TopicGroup {id:maxId + idx + 1})
SET g.descriptions = [t.description]
WITH g, t
MERGE (t)-[:IN_GROUP]->(g)
""")

""


# Clean up

In [236]:
gds.graph.drop(g_topics)

graphName                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                topics
database                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [237]:
gds.graph.drop(g_topics2)

graphName                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       reweight_topics
database                                                                